In [1]:
import os
import os.path as osp
import shutil
import pandas as pd
import numpy as np
import os
from glob import glob
from functools import reduce

import torch

val_labels = torch.cat([el.target.view(-1,30) for el in torch.load('./dataset/converted/val.pt',map_location='cuda')],
                 axis=0)
print(val_labels.shape)

def get_prediction_dataframes():
    """
        Reads the csvs of all models, concatenates them into a single one.
        
        Returns:
            -Concatenated predictions for the validation set
            -Concatenated predictions for the test set
            -Names of all the models
        
    """
    test_fname = [y for x in os.walk('./preds') for y in glob(os.path.join(x[0], '*_test.csv'))]
    _suffix_len = len('_test.csv')

    model_names = [osp.basename(r)[0:-_suffix_len] for r in test_fname]
    val_fname = [osp.join(osp.dirname(r),osp.basename(r)[0:-_suffix_len]+"_val.csv") for r in test_fname]

    """ Val """
    val_preds = np.array([pd.read_csv(f,header=None) for f in val_fname])
    val_preds = torch.transpose(torch.FloatTensor(val_preds).cuda(),0,2).transpose(0,1)
    
    test_preds = np.array([pd.read_csv(f,header=None) for f in test_fname])
    test_preds = torch.transpose(torch.FloatTensor(test_preds).cuda(),0,2).transpose(0,1)

    return val_preds, test_preds, model_names

val_preds, test_preds, model_names = get_prediction_dataframes()
         
print(model_names)

torch.Size([36938, 30])
['LSTM6', 'RNN_And_Att2', 'Lightgbm_3', 'LSTM5', 'LSTM8', 'Conv1d_1', 'RNN_And_Att', 'LSTM7', 'Lightgbm']


In [2]:
print(val_labels.shape)
print(val_preds.shape)
print(test_preds.shape)

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
class ConstrainedLinear(torch.nn.Linear):



    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.mix_w = Parameter(torch.ones(2))
        self.rbf_sigma_squared = Parameter(torch.ones(1))
        self.dist_to_mean = None

    def forward(self, inp: Tensor) -> Tensor:
        self.normalized_weight = torch.abs(self.weight)/torch.sum(torch.abs(self.weight))
        res = F.linear(inp, self.normalized_weight, 0.01*self.bias)
        
        res = torch.clip(res,0,1)
        res = res.squeeze(-1)/(res.sum(axis=1))
        return res

torch.Size([36938, 30])
torch.Size([36938, 30, 9])
torch.Size([551472, 30, 9])


In [4]:
num_iter = 1000
preds = val_preds
lin = ConstrainedLinear(in_features=val_preds.shape[-1],out_features=1).cuda()
optimizer = torch.optim.Adam(lin.parameters(), lr=1e-3)
def custom_MSE(preds,actuals):
    actuals = actuals.float()
    return 30.0*torch.mean(torch.square(preds-actuals))

for i in range(num_iter):
    optimizer.zero_grad()
    p = lin(val_preds)
    p = torch.cumsum(p,dim=1).float()
    #mean_diff = (val_labels.mean(axis=0).view(1,-1) - p.mean(axis=0).view(1,-1))
    #p += mean_diff
    loss = torch.mean(custom_MSE(p,val_labels))
    print(loss.clone())
    loss.backward()
    optimizer.step()
print(list(zip(model_names,list(lin.normalized_weight.cpu().clone().detach().numpy()[0]) )))

tensor(3.4144, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4143, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4142, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4141, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4140, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4140, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4139, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4138, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4137, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4137, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4136, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4135, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4135, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4134, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4133, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4133, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4132, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4131, device='cuda:0',

tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4091, device='cuda:0',

tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0',

tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0',

tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0', grad_fn=<CloneBackward>)
tensor(3.4085, device='cuda:0',

In [ ]:
out_df = pd.DataFrame(np.round(lin(test_preds).cpu().detach().numpy(),decimals=4))
print(out_df.shape)
display(out_df)
out_df.to_csv(f'./ensemble_3.csv.gz',index=False,header=False,compression='gzip',float_format='{:f}'.format)